In [1]:
# Import necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import random

# Import EMA Workbench modules
from ema_workbench import (ReplicatorModel, RealParameter, BooleanParameter, IntegerParameter, Constant, TimeSeriesOutcome, perform_experiments, save_results, ema_logging)

# Initialize logger to keep track of experiments run
ema_logging.log_to_stderr(ema_logging.INFO)

# Import Mesa virus on network model
import abm_model_ema

/opt/homebrew/Caskroom/miniforge/base/envs/teacos-adapter-arm/lib/python3.10/site-packages/ema_workbench/em_framework/evaluators.py:58: UserWarning: ipyparallel not installed - IpyparalleEvaluator not available
  warnings.warn("ipyparallel not installed - IpyparalleEvaluator not available")


In [2]:
def ABModel(radius = 1,
            budget = 1,
            PVPanelBatchSize = 1,
            CostOfPVPanel = 1,
            steps = 12):

    import abm_model_ema

    my_model = abm_model_ema.TholenModel(radius=radius,
                                budget=budget,
                                PVPanelBatchSize=PVPanelBatchSize,
                                CostOfPVPanel=CostOfPVPanel)


    my_model.run_model(steps)

    outcomes = my_model.datacollector.get_model_vars_dataframe()

    return {'TIME' : list(range(steps)),
            "Number of Agents that bought Solar Panels" : outcomes["Agents With Solar Panels"].tolist(),
            "Number of Agents that haven't bought Solar Panels" : outcomes["Agents Without Solar Panels"].tolist(),
            "Number of Solar Panels Bought" : outcomes["Solar Panels Bought"].tolist()}

In [3]:
# Parameters
radius = 0.0075
budget = 1000
PVPanelBatchSize = 2
CostOfPVPanel = 500
steps = 12


results = ABModel(radius = radius,
                    budget = budget,
                    PVPanelBatchSize = PVPanelBatchSize,
                    CostOfPVPanel = CostOfPVPanel,
                    steps = steps)

results

/opt/homebrew/Caskroom/miniforge/base/envs/teacos-adapter-arm/lib/python3.10/site-packages/shapely/constructive.py:181: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(
/opt/homebrew/Caskroom/miniforge/base/envs/teacos-adapter-arm/lib/python3.10/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)


12858.457706953499


{'TIME': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 'Number of Agents that bought Solar Panels': [0,
  0,
  0,
  7,
  7,
  7,
  13,
  13,
  13,
  29,
  29,
  29],
 "Number of Agents that haven't bought Solar Panels": [112,
  112,
  112,
  105,
  105,
  105,
  99,
  99,
  99,
  83,
  83,
  83],
 'Number of Solar Panels Bought': [0,
  0,
  0,
  14,
  14,
  14,
  26,
  26,
  26,
  58,
  58,
  58]}

In [4]:
# Instantiate and pass the model
model = ReplicatorModel('Tholen', function=ABModel)

In [5]:
# Define model parameters and their ranges to be sampled
model.uncertainties = [RealParameter("radius", 0.0075, 0.00005),
                       RealParameter("budget", 750, 1500),
                       RealParameter("CostOfPVPanel", 350, 650)
                       ]

model.constants = [Constant("PVPanelBatchSize", 2)]

# Define model outcomes
model.outcomes = [TimeSeriesOutcome('TIME'),
                  TimeSeriesOutcome("Number of Agents that bought Solar Panels"),
                  TimeSeriesOutcome("Number of Agents that haven't bought Solar Panels"),
                  TimeSeriesOutcome("Number of Solar Panels Bought")
                  ]

# Define the number of replications
model.replications = 10

In [6]:
# Run experiments with the aforementioned parameters and outputs
results = perform_experiments(models=model, scenarios=500)

[MainProcess/INFO] performing 500 scenarios * 1 policies * 1 model(s) = 500 experiments
  0%|                                                  | 0/500 [00:00<?, ?it/s][MainProcess/INFO] performing experiments sequentially


EMAError: some fatal error has occurred while running the experiments, not all runs have completed. expected 500, got 0